In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import time

### 用于显示爬取进度的函数

In [ ]:
# -*- coding: UTF-8 -*-
# https://blog.csdn.net/u013832707/article/details/73608504
import sys

class ShowProcess():
    """
    显示处理进度的类
    调用该类相关函数即可实现处理进度的显示
    """
    i = 0 # 当前的处理进度
    max_steps = 0 # 总共需要处理的次数
    max_arrow = 50 #进度条的长度
    infoDone = 'done'

    # 初始化函数，需要知道总共的处理次数
    def __init__(self, max_steps, infoDone = 'Done'):
        self.max_steps = max_steps
        self.i = 0
        self.infoDone = infoDone

    # 显示函数，根据当前的处理进度i显示进度
    # 效果为[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
    def show_process(self, i=None):
        if i is not None:
            self.i = i
        else:
            self.i += 1
        num_arrow = int(self.i * self.max_arrow / self.max_steps) #计算显示多少个'>'
        num_line = self.max_arrow - num_arrow #计算显示多少个'-'
        percent = self.i * 100.0 / self.max_steps #计算完成进度，格式为xx.xx%
        process_bar = '[' + '>' * num_arrow + '-' * num_line + ']'\
                      + '%.2f' % percent + '%' + '\r' #带输出的字符串，'\r'表示不换行回到最左边
        sys.stdout.write(process_bar) #这两句打印字符到终端
        sys.stdout.flush()
        if self.i >= self.max_steps:
            self.close()

    def close(self):
        print('')
        print(self.infoDone)
        self.i = 0


### 爬取漫画图片的函数

为避免相关法律问题，我就不放出该漫画网站的实际地址了。且由于网址可能会变动，需要根据实际需要进行修改。

In [ ]:
def comic_catch(url):
    ####################################
    ####################################
    print(url)
    # Make a request to the webpage
    response = requests.get(url)
    # Parse the HTML content of the webpage
    soup = BeautifulSoup(response.content, 'html.parser')
    ####################################
    ####################################
    # 获取漫画名
    comic_name=str(soup.find_all('title')).split('>')[1].split(' -')[0]
    print(comic_name)
    ####################################
    ####################################
    # 建立路径 存储下载的漫画
    if not os.path.exists('{}'.format(comic_name)):
        os.makedirs('{}'.format(comic_name))
    ####################################
    ####################################
    # 获取漫画页数
    comic_page=str(soup.find_all('label')[1]).split('：')[1].split('P')[0]
    comic_page=int(comic_page)
    #print(comic_page)
    ####################################
    ####################################
    # 获取漫画起始页地址
    liTags=soup.find_all('div', attrs={'class': "pic_box tb"})[0]
    liTags=str(liTags).split('a href="')[1].split('">')[0]
    url_pic='漫画网站主页地址'+liTags
    #print(url_pic)
    url_befor_download=url_pic
    ####################################
    ####################################
    # # 下载漫画
    process_bar = ShowProcess(comic_page, 'Processing Finished') # 进度条
    for i in range(comic_page):
        # 下载并获取下一页地址
        # Make a request to the webpage
        response = requests.get(url_befor_download)
        # Parse the HTML content of the webpage
        soup = BeautifulSoup(response.content, 'html.parser')
        # Find all image tags in the HTML
        img_tags = soup.find_all('img')
        # Find all image tags in the HTML
        img_tags = soup.find_all('img')
        # 文件名-页码
        page_name = (img_tags[2])['alt']
        #print(page_name)
        if not os.path.exists(comic_name+'/' + page_name+'.jpg'):
            # 文件内容-图片下载地址
            download_page = (img_tags[2])['src']
            download_page='https:'+download_page
            #print(download_page)
            img_data = requests.get(download_page).content
            with open(comic_name+'/' + page_name+'.jpg', 'wb') as handler:
                handler.write(img_data)
        url_befor_download=soup.find_all('link')
        url_befor_download=url_befor_download[4]['href']    
        url_befor_download='漫画网站主页地址'+url_befor_download 
        process_bar.show_process() # 进度条
    # print('sleep 10s')
    # time.sleep(10)
    # print('wake up')
    



### 一次性爬取多本漫画

提前将 漫画详情页链接地址 存放在 comic_catch_list.txt 文件中，一行存储一个链接地址

In [ ]:
f=open('comic_catch_list.txt','r') 
# 读取所有漫画链接
comic_list=[]
for i in f:
    comic_list.append(i.strip('\n'))
f.close()
# 开始下载
for i in comic_list:
    comic_catch(i)
    

### 有时会遇到下面的报错

SSLError: HTTPSConnectionPool(host='漫画网站主页地址', port=443): Max retries exceeded with url: //photos-view-id-17204800.html (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))

这种报错可能是网络状况不佳导致的

### 只爬取一本漫画

可能会遇到无法作为文件夹名字的漫画名，此时需要用合适的文件夹名，手动建立存储漫画的文件夹，建议采用下方对单本漫画的下载方法

In [ ]:

####################################
####################################
url='漫画详情页链接地址'
# Make a request to the webpage
response = requests.get(url)
# Parse the HTML content of the webpage
soup = BeautifulSoup(response.content, 'html.parser')
####################################
####################################
# # 获取漫画名
# comic_name=str(soup.find_all('title')).split('>')[1].split(' -')[0]
# print(comic_name)
# ####################################
# ####################################
# # 建立路径 存储下载的漫画
# if not os.path.exists('{}'.format(comic_name)):
#     os.makedirs('{}'.format(comic_name))
####################################
####################################
# 获取漫画页数
comic_page=str(soup.find_all('label')[1]).split('：')[1].split('P')[0]
comic_page=int(comic_page)
#print(comic_page)
####################################
####################################
# 获取漫画起始页地址
liTags=soup.find_all('div', attrs={'class': "pic_box tb"})[0]
liTags=str(liTags).split('a href="')[1].split('">')[0]
url_pic='漫画网站主页地址'+liTags
#print(url_pic)
url_befor_download=url_pic
####################################
####################################
# # 下载漫画
process_bar = ShowProcess(comic_page, 'Processing Finished') # 进度条
for i in range(comic_page):
    # 下载并获取下一页地址
    # Make a request to the webpage
    response = requests.get(url_befor_download)
    # Parse the HTML content of the webpage
    soup = BeautifulSoup(response.content, 'html.parser')
    # Find all image tags in the HTML
    img_tags = soup.find_all('img')
    # Find all image tags in the HTML
    img_tags = soup.find_all('img')
    # 文件名-页码
    page_name = (img_tags[2])['alt']
    #print(page_name)
    if not os.path.exists('漫画名'+'/' + page_name+'.jpg'):
        # 文件内容-图片下载地址
        download_page = (img_tags[2])['src']
        download_page='https:'+download_page
        #print(download_page)
        img_data = requests.get(download_page).content
        with open('漫画名'+'/' + page_name+'.jpg', 'wb') as handler:
            handler.write(img_data)
    url_befor_download=soup.find_all('link')
    url_befor_download=url_befor_download[4]['href']    
    url_befor_download='漫画网站主页地址'+url_befor_download
    process_bar.show_process() # 进度条

